In [1]:
import pandas as pd

encoded_data = pd.read_csv('encoded_data.csv')
encoded_data.head()

,Customer Name,uid,business start date,business duration (years),Normalized Performance Index,business sector_Agriculture,business sector_Fisheries,business sector_Hospitality,business sector_IT,business sector_Manufacturing,...,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000
0,Alice Sarah,9412e6f7-44a7-4268-8499-018a7bf9951a,2020-03-10,3.61,0.569356,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,William Namwalwa,afb7267f-ae22-4c74-a206-c3f274548e74,2014-04-08,9.54,0.640447,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,Alice Florence,dca85456-0e9f-43f1-8c7c-62fff8691106,2022-12-14,0.85,0.576399,0,0,0,1,0,...,1,1,0,0,0,1,0,0,0,0
3,Sarah William,9223b5d4-88f8-468e-9931-6070a11ff58e,2005-05-09,18.46,0.514231,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,0
4,Namwalwa Alice,464f9a6f-0573-44de-8e6e-4bfb0ee77298,2017-01-19,6.75,0.567989,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0


## Use Kmeans Clusters to create clusters

In [2]:
from sklearn.cluster import KMeans

encoded_data_cleaned = encoded_data.drop(columns=['Customer Name', 'uid', 'business start date'])

# apply kmeans
kmeans = KMeans(n_clusters=5, random_state=42)

#create a new column called cluster and add the kmeans clusters into that column
encoded_data_cleaned['cluster'] = kmeans.fit_predict(encoded_data_cleaned) 

# update the orgiginal clustering_data_df with the cluster column since we need the originial df to perfom the merge with the customer_data
# (need the uid)
encoded_data['cluster'] = encoded_data_cleaned['cluster'] 

encoded_data.head(3)

/Users/alice/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,Customer Name,uid,business start date,business duration (years),Normalized Performance Index,business sector_Agriculture,business sector_Fisheries,business sector_Hospitality,business sector_IT,business sector_Manufacturing,...,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000,cluster
0,Alice Sarah,9412e6f7-44a7-4268-8499-018a7bf9951a,2020-03-10,3.61,0.569356,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,3
1,William Namwalwa,afb7267f-ae22-4c74-a206-c3f274548e74,2014-04-08,9.54,0.640447,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,2
2,Alice Florence,dca85456-0e9f-43f1-8c7c-62fff8691106,2022-12-14,0.85,0.576399,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,3


## Cluster Analysis
Given the 5 clusters we have created, we can now start analysis to answer our business question. 
ToDo later
